# Simulations de résultat d'élection au Grand Conseil valaisan

Outil de simulation de résultats, se basant sur les résultats communaux réels d'une année électorale donnée (par ex. 2017 - fichier de résultats obtenu et formaté via utilisation des scripts _get-results.ipynb_ et _augment-results.ipynb_), et permettant de faire varier le contexte de vote :

- nombre de députés (130, 100, ...)
- nombre de circonscriptions électorales (13, 6, 3, 1, ...)
- base de calcul pour la répartition régionale des sièges (population suisse, population totale, ...)
- quorum (8%, 3%, ...)

## Fonctions

In [57]:
import pandas as pd
import requests


# Calcule le résultat de l'élection pour un quorum donné. Concrètement, 
# regroupe par district, somme les suffrages partisans, vérifie le quorum,
# attribue les sièges (plus gros restes)

# Arguments : 
# - df = dataframe chargée, par ex. depuis 2017-resultats-communes-augmente.xlsx
#       -> une ligne par commune
#       -> "District" = nom de la circonscription électorale retenue
#       -> "Electorat " + nom du parti = estimation de l'électorat du parti dans la commune
#       -> 'Sièges' = nombre de sièges pour le "district" (circonscription) (calculé auparavant avec setDistrictsRepartition)
# - quorum (entre 0 et 1. Typiquement 0.08 ou 0.03)
# - outputfilename -> nom du fichier où écrire résultats par district
#  

# écrit les résultats par district dans des Excel (TODO)
# retourn un dict avec les résultats partisans uniquement

def computeResults(df, quorum, outputfilename):
    
    # pour éviter de mauvaises surprises...
    pd.options.mode.chained_assignment = 'raise'
    
    # récupérer noms des partis (utile)
    parties = [c[len('Electorat '):] for c in df.columns if ('Electorat' in c)]
    
    # filtrer les colonnes pertinentes
    electorat = [c for c in df.columns if ('Electorat' in c)]
    autres = ['District','Sièges','Electeurs inscrits','Votants','Bulletins blancs', 'Bulletins nuls',
           'Bulletins valables']
    df = df[autres+electorat].copy()
    
    # 1. calcul des suffrages - estimation : électorat de base x nombre de sièges pour circonscription
    for p in parties:
        df['Suffrages '+p] = df.apply(lambda row: row['Electorat '+p] * row['Sièges'],axis=1)

    # 2. aggrégation des scores par "district"
    # get all columns with scores / number of votes, i.e. all except Sièges, District, Participation 
    cols = [c for c in df.columns if (c != 'Sièges') and (c != 'District') and (c != 'Participation (%)')]
    # aggregation rule for those columns : sum
    aggreg = {c:"sum" for c in cols} # all 
    # aggregation rule for 'sièges' column : take first value (don't sum)
    aggreg.update({'Sièges':'first'})
    # group by district + aggregate by summing scores over the district
    grp = df.groupby('District')[cols+['Sièges']]
    result = grp.agg(aggreg)
    result = result.reset_index()
    #result
        
    # 3. calcul de l'attribution des sièges par "district"
    # suffrages partisans dans le district
    cols = ['Suffrages '+c for c in parties]
    result['Somme suffrages'] = result[cols].sum(axis=1)
   
    # 3.1 quorum -> mettres suffrages à 0 si quorum pas atteint !
    def applyQuorum(suffrages_col,row,quorum):
        if row['Somme suffrages'] == 0:
            return 0 # cas où une circonscription a 0 sièges (ou personne n'a voté)
        elif row[suffrages_col] / row['Somme suffrages'] < quorum:
            return 0
        else:
            return row[suffrages_col]
        
    for p in parties:
        result['Suffrages '+p] = result.apply(lambda x: applyQuorum('Suffrages '+p,x,quorum),axis=1)
        
    # recalculer somme des suffrages après filtrage des formations éliminées par quorum
    result['Somme suffrages'] = result[cols].sum(axis=1)

    # 3.2 calculer quotient (= nombre de sièges automatiques) + reste
    for p in parties:
        result['Quotient '+p] = (result['Sièges'] * result['Suffrages '+p]).floordiv(result['Somme suffrages'])
        result['Modulo '+p] = (result['Sièges'] * result['Suffrages '+p]).mod(result['Somme suffrages'])
    
    # 3.3 création d'un 'dict' avec les scores par parti par district pour 
    # appliquer répartition des sièges restants selon plus gros restes
    # -> extraire les infos utiles de la dataframe pour la suite du processus, 
    # à savoir uniquement district, sièges, quotient et modulo par parti
    cols = ['District', 'Sièges'] + ['Quotient '+c for c in parties] + ['Modulo '+c for c in parties]
    repart = result[cols].to_dict('index')
    
    # 3.4 répartition selon méthode plus gros restes
    # répartit selon la méthode des plus gros restes les sièges restants dans le "district" (circonscription) donné
    # et reformate le résultat ('sièges') par parti. Attention : effets de bord sur l'argument (ajout de champs)
    def repartition(r):
        # calculer sièges déjà attribués
        s = 0
        for key,value in r.items():
            if 'Quotient ' in key:
                s += value
        # s'il reste des sièges -> les attribuer
        if(r['Sièges'] > s):
            # trouver le modulo le plus haut
            mods = {key:value for key,value in r.items() if 'Modulo' in key}
            sorted_mods = sorted(mods, key=mods.get, reverse=True)
            for i in range(0,int(r['Sièges']-s)):
                extra = sorted_mods[i]
                party = extra[len('Modulo '):]
                r['Extra '+party] = 1
        #return r
        out = {}
        for p in parties:
            out['Sièges '+p]=r['Quotient '+p]
            #print(r.keys())
            if ('Extra '+p) in list(r.keys()):
                out['Sièges '+p] += r['Extra '+p]
        out['District'] = r['District']
        return out
    # calculer nombre de sièges pour tous les "districts", en faire une dataframe
    final_result = pd.DataFrame.from_dict([ repartition(value) for key,value in repart.items() ])
    
    # resultat à retourner -> return somme sieges par parti as dict
    return_result = final_result.sum().to_dict() 
    
    # ajouter somme par ligne et colonne dans le resultat final
    # + pourcentages
    #####################################################
    ######### TODO
    
    # ajouter write results as Excel
    with pd.ExcelWriter(outputfilename) as writer:
        final_result.to_excel(writer)
    
    # that's all folks -> 
    pd.options.mode.chained_assignment = 'warn'
    return return_result

In [63]:
#### CALCUL NOMBRE DE SIEGES PAR "DISTRICT" (CIRCONSCRIPTION)

# Argument : districts de la forme { 'nom_district': électeurs },
# par ex. {'dis1': 54923, 'dis2': 77943, 'dis3': 16973, 'dis4': 38813, 'dis5': 25199}
# retourne nombre de sièges par ex. {'dis1': 34, 'dis2': 47, 'dis3': 10, 'dis4': 24, 'dis5': 15}

def setSieges(districts, num_deputes=130):
    # total suffrages
    s = sum(districts.values()) 
    # sièges attribués 1ère répartition
    quotients = { key : num_deputes*value // s for key,value in districts.items() } 
    # restes division entière 
    mods = { key: num_deputes*value % s for key,value in districts.items() } 
    # sièges attribués -> init avec valeur quotient
    sieges = { key: value for key, value in quotients.items() }  
    # trier par ordre de plus gros restes et appliquer autant de fois qu'il reste de sièges à attribuer
    sorted_mods = sorted(mods, key=mods.get, reverse=True) 
    for i in range(0, num_deputes-sum(quotients.values())):
        plusgros = sorted_mods[i]
        sieges[plusgros] += 1
    return sieges



# répartit les communes dans leur "district" (=circonscriptions) + répartit le nombre de sièges auxquels 
# chaque district a droit selon un critère proportionnel à la colonne col_proportion
# Arguments : 
#  - map_district = map {'nom_district(13)': 'nom_circonscription'}, défaut rien (= 13 districts de base)
#  - map_commune = map entre {'nom_commune': 'nom_circonscription'}, défaut rien (= 13 districts de base)
#  - dataframe = celle qui est lue du fichier de vote par ex. '2017-resultats-communes.xlsx'
#  - col_proportion : en proportion de quelle colonne calculer la répartition (défaut 'Electeurs Inscrits')

def setDistrictsRepartition(dataframe, num_deputes=130, col_proportion='Electeurs inscrits', map_districts={}, map_communes={}):
    
    # attribue les circonscriptions (colonne "District") par district selon map (si existe)
    if len(map_districts) > 0:
        dataframe['District'] = dataframe['District'].apply(lambda x: map_districts[x])
    
    # attribue les circonscriptions (colonne "Districts) par commune selon map (si existe)
    if len(map_communes) > 0:
        dataframe['District'] = dataframe.apply(lambda x: map_communes[x.name],axis=1)
    
    # récupérer nombre d'électeurs inscrits par district
    grp = dataframe.groupby('District')[col_proportion].sum()
    
    # tmp = {'dis1': 54923, 'dis2': 77943, 'dis3': 16973, 'dis4': 38813, 'dis5': 25199}
    tmp = grp.to_dict() 
    
    # sieges = {'dis1': 34, 'dis2': 47, 'dis3': 10, 'dis4': 24, 'dis5': 15}
    sieges = setSieges(tmp, num_deputes)
    print("Sièges par district "+ str(sieges))
    dataframe['Sièges'] = dataframe['District'].apply(lambda x: sieges[x])
    return dataframe

In [72]:
# Calcule un scénario donné.

# Arguments : 
# - inputfile : fichier avec communes + électorats partisans
# - outputdir : répertoire dans lequel écrire résultats détaillés par district
# - nickname : surnom du scénario (par ex. sur quelle base les circonscriptions sont faites)
# - col_proportion : colonne dans inputfile à prendre pour calculer proportionnellement la répartition des sièges par district
# - map_districts : mapping entre nom des districts historiques (13) et nom de la circonscription à laquelle les attribuer (défaut vide -> prendre 13 districts historiques)
# - map_communes : idem que map_districts, mais sur la base des communes (remplace map_districts si les deux sont donnés) (défaut vide -> prendre 13 districts historiques)
# - num_deputes : nombre de députés (130 par défaut)
# - quorum (0.08 = 8% par défaut)
#

def scenario(inputfile,outputdir,nickname,col_proportion,map_districts={},map_communes={},num_deputes=130,quorum=0.08):
    
    df = pd.read_excel(inputfile)
    
    # nom scenario -> par ex. 
    num_circ = len(df['District'].unique()) # by default -> 13 districts
    if len(map_communes)>0:
        num_circ = len(set(map_communes.values()))
    elif len(map_districts)>0:
        num_circ = len(set(map_districts.values()))
    scenario = nickname + "-N"+str(num_deputes) + "-C" + str(num_circ) + "-P"+col_proportion + "-Q"+str(quorum*100)
    outputfile = outputdir + scenario + ".xlsx"
    
    # calcul nombre de sièges par district + résultat de l'élection
    df = setDistrictsRepartition(df, num_deputes=num_deputes, col_proportion=col_proportion, map_districts=map_districts, map_communes=map_communes)
    res = computeResults(df, quorum, outputfile)
    res['Scénario']=scenario
    
    return res

## Scénarios

In [79]:
# Scénario : chaque commune est une circonscription 
# (mais certaines sont quand même trop petites pour avoir des sièges)!
scenario('2017-resultats-communes-augmente.xlsx',
         './2017-scenarios/',
         'Communes',
         num_deputes=100,
         map_communes=d['Commune'].to_dict(), # chaque commune est une circonscription
         col_proportion='Suisses',
         quorum=0.08)

Sièges par district {'Evolène': 1, 'Lalden': 0, 'Visp': 2, 'Evionnaz': 0, 'Agarn': 0, 'Grächen': 1, 'Trient': 0, 'Saint-Martin': 0, 'Naters': 3, 'Salgesch': 1, 'Ayent': 1, 'Eggerberg': 0, 'Vex': 1, 'Charrat': 1, 'Vérossaz': 0, 'Bitsch': 0, 'Martigny-Combe': 1, 'Vernayaz': 1, 'Varen': 0, 'Chalais': 1, 'Nendaz': 2, 'Steg-Hohtenn': 1, 'Ried-Brig': 1, 'Ausserberg': 0, 'Mont-Noble': 0, 'Saint-Maurice': 1, 'Simplon': 0, 'Ergisch': 0, 'Liddes': 0, 'Goms': 0, 'Chippis': 0, 'Staldenried': 0, 'Baltschieder': 1, 'Törbel': 0, 'Turtmann-Unterems': 0, 'Leukerbad': 0, 'Bettmeralp': 0, 'Bourg-Saint-Pierre': 0, 'Guttet-Feschel': 0, 'Vionnaz': 1, 'Anniviers': 1, 'Kippel': 0, 'Saillon': 1, 'Embd': 0, 'Saas-Balen': 0, 'Vétroz': 2, 'Zwischbergen': 0, 'Grimisuat': 1, 'Saas-Grund': 0, 'Binn': 0, 'Massongex': 1, 'Raron': 1, 'Termen': 0, 'Finhaut': 0, 'Saas-Almagell': 0, 'Port-Valais': 1, 'Orsières': 1, 'Randa': 0, 'Saxon': 2, 'St. Niklaus': 1, 'Sierre': 4, 'Collombey-Muraz': 3, 'Wiler (Lötschen)': 0, 'Veysonn

{'District': "AgarnAlbinenAnniviersArbazArdonAusserbergAyentBagnesBaltschiederBellwaldBettmeralpBinnBisterBitschBlattenBourg-Saint-PierreBovernierBrig-GlisBürchenChalaisChamosonChampéryCharratChippisCollombey-MurazCollongesContheyCrans-MontanaDorénazEggerbergEischollEistenEmbdErgischErnenEvionnazEvolèneFerdenFieschFieschertalFinhautFullyGampel-BratschGomsGrengiolsGrimisuatGrächenGrôneGuttet-FeschelHérémenceIcogneIndenIsérablesKippelLaldenLaxLensLeukLeukerbadLeytronLiddesMartignyMartigny-CombeMassongexMiègeMont-NobleMontheyMörel-FiletNatersNendazNiedergestelnOberemsObergomsOrsièresPort-ValaisRandaRaronRiddesRied-BrigRiederalpSaas-AlmagellSaas-BalenSaas-FeeSaas-GrundSaillonSaint-GingolphSaint-LéonardSaint-MartinSaint-MauriceSalgeschSalvanSavièseSaxonSembrancherSierreSimplonSionSt. NiklausStaldenStaldenriedSteg-HohtennTermenTrientTroistorrentsTurtmann-UnteremsTäschTörbelUnterbächVal-d'IlliezVarenVenthôneVernayazVexVeyrasVeysonnazVionnazVispVisperterminenVollègesVouvryVérossazVétrozWiler (

In [80]:
sieges = {'Evolène': 1, 'Lalden': 0, 'Visp': 2, 'Evionnaz': 0, 'Agarn': 0, 'Grächen': 1, 'Trient': 0, 'Saint-Martin': 0, 'Naters': 3, 'Salgesch': 1, 'Ayent': 1, 'Eggerberg': 0, 'Vex': 1, 'Charrat': 1, 'Vérossaz': 0, 'Bitsch': 0, 'Martigny-Combe': 1, 'Vernayaz': 1, 'Varen': 0, 'Chalais': 1, 'Nendaz': 2, 'Steg-Hohtenn': 1, 'Ried-Brig': 1, 'Ausserberg': 0, 'Mont-Noble': 0, 'Saint-Maurice': 1, 'Simplon': 0, 'Ergisch': 0, 'Liddes': 0, 'Goms': 0, 'Chippis': 0, 'Staldenried': 0, 'Baltschieder': 1, 'Törbel': 0, 'Turtmann-Unterems': 0, 'Leukerbad': 0, 'Bettmeralp': 0, 'Bourg-Saint-Pierre': 0, 'Guttet-Feschel': 0, 'Vionnaz': 1, 'Anniviers': 1, 'Kippel': 0, 'Saillon': 1, 'Embd': 0, 'Saas-Balen': 0, 'Vétroz': 2, 'Zwischbergen': 0, 'Grimisuat': 1, 'Saas-Grund': 0, 'Binn': 0, 'Massongex': 1, 'Raron': 1, 'Termen': 0, 'Finhaut': 0, 'Saas-Almagell': 0, 'Port-Valais': 1, 'Orsières': 1, 'Randa': 0, 'Saxon': 2, 'St. Niklaus': 1, 'Sierre': 4, 'Collombey-Muraz': 3, 'Wiler (Lötschen)': 0, 'Veysonnaz': 0, 'Leuk': 1, 'Bellwald': 0, 'Visperterminen': 1, 'Monthey': 5, 'Ferden': 0, 'Conthey': 3, 'Saint-Léonard': 1, 'Fully': 3, 'Brig-Glis': 4, 'Crans-Montana': 3, 'Isérables': 0, 'Grône': 1, 'Champéry': 0, 'Fieschertal': 0, 'Ardon': 1, 'Savièse': 3, 'Leytron': 1, 'Blatten': 0, 'Martigny': 5, 'Bovernier': 0, 'Inden': 0, 'Gampel-Bratsch': 1, 'Bister': 0, 'Albinen': 0, 'Venthône': 1, 'Riederalp': 0, 'Oberems': 0, 'Veyras': 1, 'Mörel-Filet': 0, 'Täsch': 0, 'Sembrancher': 0, 'Ernen': 0, 'Hérémence': 1, 'Vollèges': 1, 'Eisten': 0, 'Grengiols': 0, 'Fiesch': 0, 'Riddes': 1, 'Eischoll': 0, 'Niedergesteln': 0, 'Bürchen': 0, 'Lax': 0, 'Bagnes': 2, "Val-d'Illiez": 1, 'Lens': 1, 'Miège': 1, 'Obergoms': 0, 'Stalden': 0, 'Vouvry': 1, 'Troistorrents': 2, 'Arbaz': 0, 'Saas-Fee': 1, 'Chamoson': 1, 'Zeneggen': 0, 'Saint-Gingolph': 0, 'Zermatt': 1, 'Sion': 10, 'Salvan': 1, 'Unterbäch': 0, 'Icogne': 0, 'Collonges': 0, 'Dorénaz': 0}

In [81]:
[{key:sieges[key]} for key in (sorted(sieges, key=sieges.get, reverse=True))]

[{'Sion': 10},
 {'Monthey': 5},
 {'Martigny': 5},
 {'Brig-Glis': 4},
 {'Sierre': 4},
 {'Savièse': 3},
 {'Collombey-Muraz': 3},
 {'Conthey': 3},
 {'Crans-Montana': 3},
 {'Naters': 3},
 {'Fully': 3},
 {'Visp': 2},
 {'Nendaz': 2},
 {'Vétroz': 2},
 {'Saxon': 2},
 {'Bagnes': 2},
 {'Troistorrents': 2},
 {'Evolène': 1},
 {'Grächen': 1},
 {'Salgesch': 1},
 {'Ayent': 1},
 {'Visperterminen': 1},
 {'Charrat': 1},
 {'Chalais': 1},
 {'Zermatt': 1},
 {'Martigny-Combe': 1},
 {'Vernayaz': 1},
 {'Steg-Hohtenn': 1},
 {'Ried-Brig': 1},
 {'Salvan': 1},
 {'Baltschieder': 1},
 {'Lens': 1},
 {'Vionnaz': 1},
 {'Anniviers': 1},
 {'Saillon': 1},
 {'Grimisuat': 1},
 {'Chamoson': 1},
 {'Massongex': 1},
 {'Raron': 1},
 {'Orsières': 1},
 {'St. Niklaus': 1},
 {'Leuk': 1},
 {'Vex': 1},
 {'Saint-Léonard': 1},
 {'Ardon': 1},
 {'Leytron': 1},
 {'Gampel-Bratsch': 1},
 {'Port-Valais': 1},
 {'Venthône': 1},
 {'Veyras': 1},
 {'Hérémence': 1},
 {'Vollèges': 1},
 {'Riddes': 1},
 {"Val-d'Illiez": 1},
 {'Grône': 1},
 {'Miège': 

In [88]:
# scenario 1 circonscription (canton)
d = pd.read_excel('2017-resultats-communes-augmente.xlsx')
map_one_circ = {commune:'Unique' for commune,value in d['Commune'].to_dict().items() }
scenario('2017-resultats-communes-augmente.xlsx',
         './2017-scenarios/',
         'Unique',
         num_deputes=100,
         map_communes= map_one_circ, # une seule circonscription
         col_proportion='Population',
         quorum=0.03)

Sièges par district {'Unique': 100}


{'District': 'Unique',
 'Scénario': 'Unique-N100-C1-PPopulation-Q3.0',
 'Sièges CSP': 8.0,
 'Sièges CVP': 10.0,
 'Sièges PDC': 24.0,
 'Sièges PLR': 20.0,
 'Sièges PS': 14.0,
 'Sièges RCV': -0.0,
 'Sièges UDC': 17.0,
 'Sièges Verts': 7.0}

In [87]:
scenario('2017-resultats-communes-augmente.xlsx',
         './2017-scenarios/',
         'Historique',
         num_deputes=130,
         col_proportion='Suisses',
         quorum=0.08)

Sièges par district {'Hérens': 5, 'Brig': 11, 'Goms': 2, 'Monthey': 17, 'Visp': 11, 'Conthey': 11, 'Martigny': 17, 'Westlich Raron': 4, 'Leuk': 5, 'Östlich Raron': 1, 'St-Maurice': 5, 'Sion': 18, 'Sierre': 17, 'Entremont': 6}


{'District': 'BrigContheyEntremontGomsHérensLeukMartignyMontheySierreSionSt-MauriceVispWestlich RaronÖstlich Raron',
 'Scénario': 'Historique-N130-C14-PSuisses-Q8.0',
 'Sièges CSP': 9.0,
 'Sièges CVP': 14.0,
 'Sièges PDC': 31.0,
 'Sièges PLR': 26.0,
 'Sièges PS': 18.0,
 'Sièges RCV': -0.0,
 'Sièges UDC': 24.0,
 'Sièges Verts': 8.0}

In [89]:
## scénario effectif de 2017 (6 arrondissements)
arrondissements = {
    'Goms':'ArrBrig',
    'Östlich Raron':'ArrBrig',
    'Brig':'ArrBrig', 
    'Visp':'ArrVisp',
    'Westlich Raron':'ArrVisp',
    'Leuk':'ArrVisp',
    'Sierre':'ArrSierre',
    'Hérens':'ArrSion',
    'Sion':'ArrSion',
    'Conthey':'ArrSion',
    'Martigny':'ArrMy',
    'Entremont':'ArrMy',
    'St-Maurice':'ArrMon',
    'Monthey':'ArrMon'
}
scenario('2017-resultats-communes-augmente.xlsx',
         './2017-scenarios/',
         '6Arrondissements',
         num_deputes=130,
         map_districts = arrondissements,
         col_proportion='Suisses',
         quorum=0.08)

Sièges par district {'ArrMy': 23, 'ArrBrig': 15, 'ArrSion': 33, 'ArrVisp': 20, 'ArrMon': 22, 'ArrSierre': 17}


{'District': 'ArrBrigArrMonArrMyArrSierreArrSionArrVisp',
 'Scénario': '6Arrondissements-N130-C6-PSuisses-Q8.0',
 'Sièges CSP': 11.0,
 'Sièges CVP': 13.0,
 'Sièges PDC': 31.0,
 'Sièges PLR': 27.0,
 'Sièges PS': 17.0,
 'Sièges RCV': -0.0,
 'Sièges UDC': 22.0,
 'Sièges Verts': 9.0}